In [15]:
#Import the necessary libraries
import h5py
import numpy as np
import math
import json

In [16]:
#Load the hd5 files
test_data_loc = "mnist_testdata.hdf5"
network_params_loc = "mnist_network_params.hdf5"

In [17]:
#Read hd5 file
test_data_file = h5py.File(test_data_loc, "r")
network_params_file = h5py.File(network_params_loc, "r")

In [18]:
#Load data from the hd5 files
test_X = test_data_file["xdata"][:]
test_Y = test_data_file["ydata"][:]
W1 = network_params_file["W1"][:]
W2 = network_params_file["W2"][:]
W3 = network_params_file["W3"][:]
b1 = network_params_file["b1"][:]
b2 = network_params_file["b2"][:]
b3 = network_params_file["b3"][:]

In [19]:
#Define the softmax function
def softmax(X):
  summation = sum([math.exp(i) for i in X[0]])
  output = np.array([math.exp(i)/summation for i in X[0]])
  return output

In [20]:
#Define the ReLU function
def ReLU(X):
  y = np.array([max(i,0) for i in X[0]])
  return y

In [21]:
#Function to get output from Layer 1
def HiddenLayer1(X,W,b):
  n_samples, n_features = X.shape
  output = np.empty((n_samples,200))
  bias = b.reshape((1,200))
  for i in range(n_samples):
    x = X[i].reshape((1,n_features))
    a = np.dot(x,W.T) + bias
    output[i] = ReLU(a)
  return output

In [22]:
#Function to get output from Layer 2
def HiddenLayer2(X,W,b):
  n_samples, n_features = X.shape
  output = np.empty((n_samples,100))
  bias = b.reshape((1,100))
  for i in range(n_samples):
    x = X[i].reshape((1,n_features))
    a = np.dot(x,W.T) + bias
    output[i] = ReLU(a)
  return output

In [23]:
#Function to get output from Final Layer
def OutputLayer(X,W,b):
  n_samples, n_features = X.shape
  output = np.empty((n_samples,10))
  bias = b.reshape((1,10))
  for i in range(n_samples):
    x = X[i].reshape((1,100))
    a = np.dot(x,W.T) + bias
    output[i] = softmax(a)
  return output

In [24]:
#Function to calculate accuracy
def accuracy(Y,Y_predicted):
  acc = 0
  for i in range(10000):
    classifiend_value = np.argmax(Y_predicted[i])
    actual_value = np.argmax(Y[i])
    if(classifiend_value == actual_value):
      acc+=1
  return acc

In [25]:
#Implementing the Multilayer Perceptron Neural Netwrok
def MLP_NN(X,W1,b1,W2,b2,W3,b3):
  output_HL1 = HiddenLayer1(X , W1, b1)
  output_HL2 = HiddenLayer2(output_HL1, W2, b2)
  output = OutputLayer(output_HL2, W3, b3)
  return output

In [26]:
#Calling the MLP Neural Network and calculating the accuracy 
output = MLP_NN(test_X,W1,b1,W2,b2,W3,b3)
acc = accuracy(test_Y, output)
acc

9790

In [28]:
#Writing the data to a JSON file
data = []
for i in range(10000):
  ind = i
  activations = [float(k) for k in output[i]]
  classifiend_value = np.argmax(output[i])
  data.append({"index":int(i),  "activations":activations,"classification":int(classifiend_value)})
data
with open("result.json", "w") as f: 
  f.write(json.dumps(data))